# Problem 54

### Poker Hands

In the card game poker, a hand consists of five cards and are ranked, from lowest to highest, in the following way:

* High Card: Highest value card.
* One Pair: Two cards of the same value.
* Two Pairs: Two different pairs.
* Three of a Kind: Three cards of the same value.
* Straight: All cards are consecutive values.
* Flush: All cards of the same suit.
* Full House: Three of a kind and a pair.
* Four of a Kind: Four cards of the same value.
* Straight Flush: All cards are consecutive values of same suit.
* Royal Flush: Ten, Jack, Queen, King, Ace, in same suit.

The cards are valued in the order:
2, 3, 4, 5, 6, 7, 8, 9, 10, Jack, Queen, King, Ace.

If two players have the same ranked hands then the rank made up of the highest value wins; for example, a pair of eights beats a pair of fives (see example 1 below). But if two ranks tie, for example, both players have a pair of queens, then highest cards in each hand are compared (see example 4 below); if the highest cards tie then the next highest cards are compared, and so on.

The file, poker.txt, contains one-thousand random hands dealt to two players. Each line of the file contains ten cards (separated by a single space): the first five are Player 1's cards and the last five are Player 2's cards. You can assume that all hands are valid (no invalid characters or repeated cards), each player's hand is in no specific order, and in each hand there is a clear winner.

How many hands does Player 1 win?

In [1]:
import collections

Card = collections.namedtuple('Card', 'num suit')

In [2]:
dict_card_word_to_number = {k: idx for idx, k in enumerate('23456789TJQKA', start=2)}
print dict_card_word_to_number

dict_card_number_to_word = {v:k for k, v in dict_card_word_to_number.iteritems()}

def to_Card(cards):
    for card in cards:
        yield Card(dict_card_word_to_number[card[0]], card[1])
        
def to_word(cards):
    res = []
    for card in cards:
        res.append(dict_card_number_to_word[card.num] + card.suit)
    return res

{'A': 14, 'K': 13, 'J': 11, 'Q': 12, '3': 3, '2': 2, '5': 5, '4': 4, '7': 7, '6': 6, '9': 9, '8': 8, 'T': 10}


In [3]:
def get_numbers(cards):
    return [c.num for c in cards]

def get_suits(cards):
    return [c.suit for c in cards]

def counter_numbers(cards):
    return collections.Counter(get_numbers(cards))

def counter_suits(cards):
    return collections.Counter(get_suits(cards))

def max_number(cards):
    return max(cards, key=lambda c: c.num).num

In [4]:
def HighestCard(cards):
    
    """ Highest value card """
    
    return True, max_number(cards)


def OnePair(cards):
    
    """ Two cards of the same value """
    
    counts = counter_numbers(cards)
    num_most_common, count_most_common = counts.most_common(1)[0]
    
    if count_most_common == 2:
        return True, num_most_common
    else:
        return False, None


def TwoPairs(cards):
    
    """ Two different pairs """
    
    counts = counter_numbers(cards)
    
    # TODO: What if counts has only 1 element?
    # DONE: Impossibile, because I would have 5 cards with the same number!
    (n1, c1), (n2, c2) = counts.most_common(2)
    
    if c1 == 2 and c2 == 2:
        return True, max(n1, n2)
    else:
        return False, None

    
def ThreeOfAKind(cards):
    
    """ Three cards of the same value """
    
    counts = counter_numbers(cards)
    n_three, count_most_common = counts.most_common(1)[0]
    
    if count_most_common == 3:
        return True, n_three
    else:
        return False, None


def Straight(cards):
    
    """ All cards are consecutive values """
    
    numbers = sorted(get_numbers(cards))
    if max(numbers) - min(numbers) == 4 and len(set(numbers)) == 5:
        return True, max(numbers)
    else:
        return False, None


def Flush(cards):
    
    """ All cards of the same suit """
    
    count_different_suits = len(set(get_suits(cards)))
    if count_different_suits == 1:
        return True, max_number(cards)
    else:
        return False, None

    
def FullHouse(cards):
    
    """ Three of a kind and a pair """
    
    counts = counter_numbers(cards)
    (n1, c1), (n2, c2) = counts.most_common(2)
    if c1 == 3 and c2 == 2:
        # The text says that the highest involved card makes the rank.
        # The real rule seems to be different: if there's a tie, first we compare the triple than the couple
        return True, max_number(cards)  
    else:
        return False, None

    
def FourOfAKind(cards):
    
    """ Four cards of the same value """
    
    counts = counter_numbers(cards)
    num_most_common, count_most_common = counts.most_common(1)[0]

    if count_most_common == 4:
        return True, num_most_common
    else:
        return False, None
    

def StraightFlush(cards):
    
    """ All cards are consecutive values of same suit """
    
    is_flush, max_card_num = Flush(cards)
    if not is_flush:
        return False, None
    else:
        is_straight, _ = Straight(cards)
        if is_straight:
            return True, max_card_num
        else:
            return False, None

        
def RoyalFlush(cards):
    
    """ Ten, Jack, Queen, King, Ace, in same suit """
    
    is_straight_flush, max_card = StraightFlush(cards)
    if is_straight_flush and max_card == dict_card_word_to_number['A']:
        return True, max_card
    else:
        return False, None

In [5]:
ranks = [RoyalFlush, StraightFlush, FourOfAKind, FullHouse, Flush, Straight, ThreeOfAKind, TwoPairs, OnePair, HighestCard]

def get_rank(cards):
    for idx_rank, Rank in enumerate(ranks):
        is_valid, max_n = Rank(cards)
        if is_valid:
            return idx_rank, max_n
        
    # Not ammissible since HighestCard always returns is_valid = True
    raise RuntimeError('Something is broken!')

In [6]:
test_hands = [
    (['2C', '3S', '4H', '5D', '3H'], ['3C', '4S', '5H', '6D', 'TC']),
    (['4C', '3S', '4H', '5D', '3H'], ['3C', '6S', '5H', '6D', '6C']),
    (['9C', 'TS', 'JH', 'QC', 'KD'], ['3C', '4C', '9C', 'AC', '2C']),
    (['9C', '9S', '9D', '9H', 'KD'], ['3C', '3D', '3H', '4H', '4C']),
    (['4C', '6C', '8C', '7C', '5C'], ['QC', 'JC', 'KC', 'TC', 'AC'])
]

test_hands = [(list(to_Card(hand_player_1)), list(to_Card(hand_player_2))) 
                                          for hand_player_1, hand_player_2 in test_hands] 

In [7]:
def find_rank(hands):
    
    for hand_1, hand_2 in hands:

        print
        print 'Hand 1:', to_word(hand_1)
        idx_rank, max_n = get_rank(hand_1)
        print ranks[idx_rank].__name__, max_n

        print
        print 'Hand 2:', to_word(hand_2)
        idx_rank, max_n = get_rank(hand_2)
        print ranks[idx_rank].__name__, max_n

In [8]:
print
print 'HighestCard Test'
print HighestCard(test_hands[0][1]) # True

print
print 'OnePair Test'
print OnePair(test_hands[0][0])     # True
print OnePair(test_hands[0][1])     # False

print
print 'TwoPairs Test'
print TwoPairs(test_hands[1][0])     # True
print TwoPairs(test_hands[0][0])     # False

print
print 'ThreeOfAKind Test'
print ThreeOfAKind(test_hands[1][1]) # True
print ThreeOfAKind(test_hands[1][0]) # False
print ThreeOfAKind(test_hands[0][1]) # False

print
print 'Straight Test'
print Straight(test_hands[2][0]) # True
print Straight(test_hands[1][1]) # False
print Straight(test_hands[0][0]) # False

print
print 'Flush Test'
print Flush(test_hands[2][1]) # True
print Flush(test_hands[2][0]) # False

print
print 'FullHouse Test'
print FullHouse(test_hands[3][1]) # True
print FullHouse(test_hands[3][0]) # False

print
print 'FourOfAKind Test'
print FourOfAKind(test_hands[3][0]) # True
print FourOfAKind(test_hands[3][1]) # False

print
print 'StraightFlush Test'
print StraightFlush(test_hands[2][1]) # False (only Flush)
print StraightFlush(test_hands[2][0]) # False (only Straight)
print StraightFlush(test_hands[4][0]) # True

print
print 'RoyalFlush'
print RoyalFlush(test_hands[4][0]) # False (only StraightFlush)
print RoyalFlush(test_hands[4][1]) # True


HighestCard Test
(True, 10)

OnePair Test
(True, 3)
(False, None)

TwoPairs Test
(True, 4)
(False, None)

ThreeOfAKind Test
(True, 6)
(False, None)
(False, None)

Straight Test
(True, 13)
(False, None)
(False, None)

Flush Test
(True, 14)
(False, None)

FullHouse Test
(True, 4)
(False, None)

FourOfAKind Test
(True, 9)
(False, None)

StraightFlush Test
(False, None)
(False, None)
(True, 8)

RoyalFlush
(False, None)
(True, 14)


In [9]:
def load_data():
    
    with open('data/p054_poker.txt') as f:
        cards = f.read().split()
        
    i = 0
    while i < len(cards):
        yield cards[i:i+5], cards[i+5:i+10]
        i += 10

In [10]:
def who_wins(hands):
    
    for hand_1, hand_2 in hands:

        print '-' * 50
        print
        print 'Hand 1:', to_word(hand_1)
        idx_rank_1, max_n_1 = get_rank(hand_1)
        print ranks[idx_rank_1].__name__, max_n_1

        print
        print 'Hand 2:', to_word(hand_2)
        idx_rank_2, max_n_2 = get_rank(hand_2)
        print ranks[idx_rank_2].__name__, max_n_2
        
        print
        winner = None
        rank_diff = idx_rank_1 - idx_rank_2
        if rank_diff < 0:
            winner = 1
        elif rank_diff > 0:
            winner = 2
        else:
            max_num_diff = max_n_1 - max_n_2
            if max_num_diff > 0:
                winner = 1
            elif max_num_diff < 0:
                winner = 2
            else:
                print 'Tie of ranks and max involved card.'
                
                # HACK: since there's only one case of that and is solvable by taking the max card in the hands,
                # I will just implement this
                
                max_1, max_2 = max_number(hand_1), max_number(hand_2)
                if max_1 > max_2:
                    winner = 1
                elif max_1 < max_2:
                    winner = 2
                else:
                    raise NotImplementedError()
            
        print 'Winner:', winner
        print
        
        yield winner

In [11]:
hands = [(list(to_Card(hand_player_1)), list(to_Card(hand_player_2))) 
                                          for hand_player_1, hand_player_2 in load_data()]

winners = list(who_wins(hands))    

--------------------------------------------------

Hand 1: ['8C', 'TS', 'KC', '9H', '4S']
HighestCard 13

Hand 2: ['7D', '2S', '5D', '3S', 'AC']
HighestCard 14

Winner: 2

--------------------------------------------------

Hand 1: ['5C', 'AD', '5D', 'AC', '9C']
TwoPairs 14

Hand 2: ['7C', '5H', '8D', 'TD', 'KS']
HighestCard 13

Winner: 1

--------------------------------------------------

Hand 1: ['3H', '7H', '6S', 'KC', 'JS']
HighestCard 13

Hand 2: ['QH', 'TD', 'JC', '2D', '8S']
HighestCard 12

Winner: 1

--------------------------------------------------

Hand 1: ['TH', '8H', '5C', 'QS', 'TC']
OnePair 10

Hand 2: ['9H', '4D', 'JC', 'KS', 'JS']
OnePair 11

Winner: 2

--------------------------------------------------

Hand 1: ['7C', '5H', 'KC', 'QH', 'JD']
HighestCard 13

Hand 2: ['AS', 'KH', '4C', 'AD', '4S']
TwoPairs 14

Winner: 2

--------------------------------------------------

Hand 1: ['5H', 'KS', '9C', '7D', '9H']
OnePair 9

Hand 2: ['8D', '3S', '5D', '5C', 'AH']
OnePair 

Hand 1: ['2H', '8C', '3D', 'AH', '4D']
HighestCard 14

Hand 2: ['TH', 'TC', '7D', '8H', 'KC']
OnePair 10

Winner: 2

--------------------------------------------------

Hand 1: ['TS', '5C', '2D', '8C', '6S']
HighestCard 10

Hand 2: ['KH', 'AH', '5H', '6H', 'KC']
OnePair 13

Winner: 2

--------------------------------------------------

Hand 1: ['5S', '5D', 'AH', 'TC', '4C']
OnePair 5

Hand 2: ['JD', '8D', '6H', '8C', '6C']
TwoPairs 8

Winner: 2

--------------------------------------------------

Hand 1: ['KC', 'QD', '3D', '8H', '2D']
HighestCard 13

Hand 2: ['JC', '9H', '4H', 'AD', '2S']
HighestCard 14

Winner: 2

--------------------------------------------------

Hand 1: ['TD', '6S', '7D', 'JS', 'KD']
HighestCard 13

Hand 2: ['4H', 'QS', '2S', '3S', '8C']
HighestCard 12

Winner: 1

--------------------------------------------------

Hand 1: ['4C', '9H', 'JH', 'TS', '3S']
HighestCard 11

Hand 2: ['4H', 'QC', '5S', '9S', '9C']
OnePair 9

Winner: 2

------------------------------------

HighestCard 14

Hand 2: ['QS', '7D', '3S', '9C', '8S']
HighestCard 12

Winner: 1

--------------------------------------------------

Hand 1: ['AH', 'QH', '3C', 'JD', 'KC']
HighestCard 14

Hand 2: ['4S', '5S', '5D', 'TD', 'KS']
OnePair 5

Winner: 2

--------------------------------------------------

Hand 1: ['9H', '7H', '6S', 'JH', 'TH']
HighestCard 11

Hand 2: ['4C', '7C', 'AD', '5C', '2D']
HighestCard 14

Winner: 2

--------------------------------------------------

Hand 1: ['7C', 'KD', '5S', 'TC', '9D']
HighestCard 13

Hand 2: ['6S', '6C', '5D', '2S', 'TH']
OnePair 6

Winner: 2

--------------------------------------------------

Hand 1: ['KC', '9H', '8D', '5H', '7H']
HighestCard 13

Hand 2: ['4H', 'QC', '3D', '7C', 'AS']
HighestCard 14

Winner: 2

--------------------------------------------------

Hand 1: ['6S', '8S', 'QC', 'TD', '4S']
HighestCard 12

Hand 2: ['5C', 'TH', 'QS', 'QD', '2S']
OnePair 12

Winner: 2

--------------------------------------------------

Hand 1: ['8S', 

TwoPairs 11

Winner: 2

--------------------------------------------------

Hand 1: ['2H', '5S', 'TH', '6S', 'TS']
OnePair 10

Hand 2: ['3S', 'KS', '3C', '5H', 'JS']
OnePair 3

Winner: 1

--------------------------------------------------

Hand 1: ['2D', '9S', '7H', '3D', 'KC']
HighestCard 13

Hand 2: ['JH', '6D', '7D', 'JS', 'TD']
OnePair 11

Winner: 2

--------------------------------------------------

Hand 1: ['AC', 'JS', '8H', '2C', '8C']
OnePair 8

Hand 2: ['JH', 'JC', '2D', 'TH', '7S']
OnePair 11

Winner: 2

--------------------------------------------------

Hand 1: ['5D', '9S', '8H', '2H', '3D']
HighestCard 9

Hand 2: ['TC', 'AH', 'JC', 'KD', '9C']
HighestCard 14

Winner: 2

--------------------------------------------------

Hand 1: ['9D', 'QD', 'JC', '2H', '6D']
HighestCard 12

Hand 2: ['KH', 'TS', '9S', 'QH', 'TH']
OnePair 10

Winner: 2

--------------------------------------------------

Hand 1: ['2C', '8D', '4S', 'JD', '5H']
HighestCard 11

Hand 2: ['3H', 'TH', 'TC', '9C'

--------------------------------------------------

Hand 1: ['AH', 'AD', 'TH', '6D', '9C']
OnePair 14

Hand 2: ['9S', 'KD', 'KS', 'QH', '4H']
OnePair 13

Winner: 1

--------------------------------------------------

Hand 1: ['QD', '6H', '9C', '7C', 'QS']
OnePair 12

Hand 2: ['6D', '6S', '9D', '5S', 'JH']
OnePair 6

Winner: 1

--------------------------------------------------

Hand 1: ['AH', '8D', '5H', 'QD', '2H']
HighestCard 14

Hand 2: ['JC', 'KS', '4H', 'KH', '5S']
OnePair 13

Winner: 2

--------------------------------------------------

Hand 1: ['5C', '2S', 'JS', '8D', '9C']
HighestCard 11

Hand 2: ['8C', '3D', 'AS', 'KC', 'AH']
OnePair 14

Winner: 2

--------------------------------------------------

Hand 1: ['JD', '9S', '2H', 'QS', '8H']
HighestCard 12

Hand 2: ['5S', '8C', 'TH', '5C', '4C']
OnePair 5

Winner: 2

--------------------------------------------------

Hand 1: ['QC', 'QS', '8C', '2S', '2C']
TwoPairs 12

Hand 2: ['3S', '9C', '4C', 'KS', 'KH']
OnePair 13

Winner: 1


Hand 1: ['TD', '2H', '7D', 'JD', 'QD']
HighestCard 12

Hand 2: ['4C', '7H', '5D', 'KC', '3D']
HighestCard 13

Winner: 2

--------------------------------------------------

Hand 1: ['4C', '3H', '8S', 'KD', 'QH']
HighestCard 13

Hand 2: ['5S', 'QC', '9H', 'TC', '5H']
OnePair 5

Winner: 2

--------------------------------------------------

Hand 1: ['9C', 'QD', 'TH', '5H', 'TS']
OnePair 10

Hand 2: ['5C', '9H', 'AH', 'QH', '2C']
HighestCard 14

Winner: 1

--------------------------------------------------

Hand 1: ['4D', '6S', '3C', 'AC', '6C']
OnePair 6

Hand 2: ['3D', '2C', '2H', 'TD', 'TH']
TwoPairs 10

Winner: 2

--------------------------------------------------

Hand 1: ['AC', '9C', '5D', 'QC', '4D']
HighestCard 14

Hand 2: ['AD', '8D', '6D', '8C', 'KC']
OnePair 8

Winner: 2

--------------------------------------------------

Hand 1: ['AD', '3C', '4H', 'AC', '8D']
OnePair 14

Hand 2: ['8H', '7S', '9S', 'TD', 'JC']
Straight 11

Winner: 2

-------------------------------------------

HighestCard 11

Winner: 2

--------------------------------------------------

Hand 1: ['QS', '9C', 'KS', 'TD', '2S']
HighestCard 13

Hand 2: ['8S', '5C', '2H', '4H', 'AS']
HighestCard 14

Winner: 2

--------------------------------------------------

Hand 1: ['TH', '7S', '4H', '7D', '3H']
OnePair 7

Hand 2: ['JD', 'KD', '5D', '2S', 'KC']
OnePair 13

Winner: 2

--------------------------------------------------

Hand 1: ['JD', '7H', '4S', '8H', '4C']
OnePair 4

Hand 2: ['JS', '6H', 'QH', '5S', '4H']
HighestCard 12

Winner: 1

--------------------------------------------------

Hand 1: ['2C', 'QS', '8C', '5S', '3H']
HighestCard 12

Hand 2: ['QC', '2S', '6C', 'QD', 'AD']
OnePair 12

Winner: 2

--------------------------------------------------

Hand 1: ['8C', '3D', 'JD', 'TC', '4H']
HighestCard 11

Hand 2: ['2H', 'AD', '5S', 'AC', '2S']
TwoPairs 14

Winner: 2

--------------------------------------------------

Hand 1: ['5D', '2C', 'JS', '2D', 'AD']
OnePair 2

Hand 2: ['9D', '3D', '4C', 

TwoPairs 4

Winner: 2

--------------------------------------------------

Hand 1: ['JD', '2H', '5C', 'AS', '6C']
HighestCard 14

Hand 2: ['QC', '4D', '3C', 'TC', 'JH']
HighestCard 12

Winner: 1

--------------------------------------------------

Hand 1: ['AC', 'JD', '3H', '6H', '4C']
HighestCard 14

Hand 2: ['JC', 'AD', '7D', '7H', '9H']
OnePair 7

Winner: 2

--------------------------------------------------

Hand 1: ['4H', 'TC', 'TS', '2C', '8C']
OnePair 10

Hand 2: ['6S', 'KS', '2H', 'JD', '9S']
HighestCard 13

Winner: 1

--------------------------------------------------

Hand 1: ['4C', '3H', 'QS', 'QC', '9S']
OnePair 12

Hand 2: ['9H', '6D', 'KC', '9D', '9C']
ThreeOfAKind 9

Winner: 2

--------------------------------------------------

Hand 1: ['5C', 'AD', '8C', '2C', 'QH']
HighestCard 14

Hand 2: ['TH', 'QD', 'JC', '8D', '8H']
OnePair 8

Winner: 2

--------------------------------------------------

Hand 1: ['QC', '2C', '2S', 'QD', '9C']
TwoPairs 12

Hand 2: ['4D', '3S', '8D',

HighestCard 14

Winner: 1

--------------------------------------------------

Hand 1: ['QC', '9C', '5S', '4H', '2H']
HighestCard 12

Hand 2: ['TD', '7D', 'AS', '8C', '9D']
HighestCard 14

Winner: 2

--------------------------------------------------

Hand 1: ['8C', '2C', '9D', 'KD', 'TC']
HighestCard 13

Hand 2: ['7S', '3D', 'KH', 'QC', '3C']
OnePair 3

Winner: 2

--------------------------------------------------

Hand 1: ['4D', 'AS', '4C', 'QS', '5S']
OnePair 4

Hand 2: ['9D', '6S', 'JD', 'QH', 'KS']
HighestCard 13

Winner: 1

--------------------------------------------------

Hand 1: ['6D', 'AH', '6C', '4C', '5H']
OnePair 6

Hand 2: ['TS', '9H', '7D', '3D', '5S']
HighestCard 10

Winner: 1

--------------------------------------------------

Hand 1: ['QS', 'JD', '7C', '8D', '9C']
HighestCard 12

Hand 2: ['AC', '3S', '6S', '6C', 'KH']
OnePair 6

Winner: 2

--------------------------------------------------

Hand 1: ['8H', 'JH', '5D', '9S', '6D']
HighestCard 11

Hand 2: ['AS', '6S', 

Winner: 2

--------------------------------------------------

Hand 1: ['8S', '6S', 'TS', '3C', '6H']
OnePair 6

Hand 2: ['8D', '5S', '3H', 'TD', '6C']
HighestCard 10

Winner: 1

--------------------------------------------------

Hand 1: ['KS', '3D', 'JH', '9C', '7C']
HighestCard 13

Hand 2: ['9S', 'QS', '5S', '4H', '6H']
HighestCard 12

Winner: 1

--------------------------------------------------

Hand 1: ['7S', '6S', 'TH', '4S', 'KC']
HighestCard 13

Hand 2: ['KD', '3S', 'JC', 'JH', 'KS']
TwoPairs 13

Winner: 2

--------------------------------------------------

Hand 1: ['7C', '3C', '2S', '6D', 'QH']
HighestCard 12

Hand 2: ['2C', '7S', '5H', '8H', 'AH']
HighestCard 14

Winner: 2

--------------------------------------------------

Hand 1: ['KC', '8D', 'QD', '6D', 'KH']
OnePair 13

Hand 2: ['5C', '7H', '9D', '3D', '9C']
OnePair 9

Winner: 1

--------------------------------------------------

Hand 1: ['6H', '2D', '8S', 'JS', '9S']
HighestCard 11

Hand 2: ['2S', '6D', 'KC', '7C', '

In [12]:
sum(1 for winner in winners if winner == 1)

376